In [1]:
from z3 import *
import itertools

s = Solver()

# Aircraft symbols
p1, a, p2, b, p3 = Ints('p1 a p2 b p3')
ac = [p1, a, p2, b, p3]

# Functions
δ = Function('δ', IntSort(), IntSort(), IntSort())
R = Function('R', IntSort(), RealSort())



# Basic constraints
for x in ac: s.add(R(x) >= 0)

for (x, y) in itertools.product(ac, ac):
    s.add(δ(x, y) >= 0)
    s.add(δ(y, x) >= 0)

# a and b are δ-identical
for x in ac:
    s.add(δ(a, x) == δ(b, x))
    s.add(δ(x, a) == δ(x, b))

# Helper max functions
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs):
    if not xs:
        raise ValueError("zmax_list requires at least one element")
    m = xs[0]
    for x in xs[1:]:
        m = If(x >= m, x, m)
    return m

# Sequences
S1 = [p1, a, p2, b, p3]
S2 = [p1, b, p2, a, p3]

# Compute T_i symbolically given a sequence
def compute_T(seq):
    T = {}
    T[seq[0]] = R(seq[0])
    for i in range(1, len(seq)):
        preds = seq[:i]
        T[seq[i]] = zmax(R(seq[i]),
                         zmax_list([T[x] + δ(x, seq[i]) for x in preds]))
    return T

T1 = compute_T(S1)
T2 = compute_T(S2)

# Compare makespans (symbolically)
makespan1 = T1[S1[-1]]
makespan2 = T2[S2[-1]]

s.add(R(a) <= R(b))
# Negate the claim to look for a counterexample:
s.add(makespan1 > makespan2)


print("Checking if S1 has a worse makespan than S2:")
print(s.check())

Checking if S1 has a worse makespan than S2:
unsat
